In [4]:
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle (1).json


In [2]:
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification


  0% 0.00/221k [00:00<?, ?B/s]
100% 221k/221k [00:00<00:00, 85.7MB/s]
 41% 5.00M/12.1M [00:00<00:00, 32.1MB/s]
100% 12.1M/12.1M [00:00<00:00, 58.9MB/s]
 99% 271M/273M [00:02<00:00, 116MB/s] 
100% 273M/273M [00:02<00:00, 118MB/s]


In [3]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 325, done.
remote: Total 325 (delta 0), reused 0 (delta 0), pack-reused 325
Receiving objects: 100% (325/325), 263.74 KiB | 874.00 KiB/s, done.
Resolving deltas: 100% (183/183), done.


In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip the file
!unzip uncased_L-12_H-768_A-12.zip

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import numpy as np 
train = pd.read_csv("train.csv.zip")
test = pd.read_csv("test.csv.zip")

df_bert = pd.DataFrame({'user_id':train['id'],
            'label':np.where(train.target > .5,1,0),
            'alpha':['a']*train.shape[0],
            'text':train["comment_text"].replace(r'\n',' ',regex=True)})

df_bert_train, df_bert_dev = train_test_split(df_bert, test_size=0.01)

# Creating test dataframe according to BERT
df_bert_test = pd.DataFrame({'User_ID':test['id'],
                 'text':test['comment_text'].replace(r'\n',' ',regex=True)})

In [0]:
df_bert_train.to_csv('data/train.tsv', sep='\t', index=False, header=False)
df_bert_dev.to_csv('data/dev.tsv', sep='\t', index=False, header=False)
df_bert_test.to_csv('data/test.tsv', sep='\t', index=False, header=True)

In [0]:
!python bert/run_classifier.py \
--task_name=cola \
--do_train=true \
--do_eval=true \
--do_predict=true \
--data_dir=data \
--vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
--bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
--init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
--max_seq_length=400 \
--train_batch_size=32 \
--learning_rate=2e-5 \
--num_train_epochs=1.0 \
--output_dir=bert_output \
--save_checkpoints_steps=9999999

BERT feature extraction

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np 
import gc
from tqdm import tqdm
train = pd.read_csv("train.csv.zip")['comment_text'].replace(r'\n', ' ', regex=True)
batch_size = 512
n_batch = len(train) // batch_size
gc.collect()

In [0]:
for batch in tqdm(range(n_batch)):
    batch_start = batch_size * batch
    batch_end = batch_size * (batch+1)
    train[batch_start:batch_end].to_csv('input.txt', header=False, index=False)
    tmp_matrix = np.zeros((batch_size, 768), dtype='float16')
    
    !python bert/extract_features.py \
    --input_file=input.txt \
    --output_file=output.jsonl \
    --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
    --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
    --init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
    --layers=-1 \
    --max_seq_length=220 \
    --batch_size=64
    
    bert_output = pd.read_json("output.jsonl", lines=True)

    for i in range(batch_size):
        tmp = np.zeros(768, )
        voc = len(bert_output['features'][i])-2
        for j in range(1, voc+1):
            tmp += np.asarray(bert_output['features'][i][j]['layers'][0]['values'], dtype='float16') / voc
        tmp_matrix[i] = tmp
    if batch > 0:
        tmp_matrix = np.vstack((tmp_matrix, np.load('drive/My Drive/Data/bert_matrix.npy')))
    np.save('drive/My Drive/Data/bert_matrix', tmp_matrix)
    del tmp_matrix, bert_output
    gc.collect()
    


  0%|          | 0/3525 [00:00<?, ?it/s]

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] " this is so cool . it ' s like , ' would you want your mother to read this ? ? ' really great idea , well done ! " [SEP]
INFO:tensorflow:input_ids: 101 1000 2023 2003 2061 4658 1012 2009 1005 1055 2066 1010 1005 2052 2017 2215 2115 2388 2000 3191 2023 1029 1029 1005 2428 2307 2801 1010 2092 2589 999 1000 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 


  0%|          | 1/3525 [01:04<62:54:05, 64.26s/it]

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] " part 2 of 2 : as you say , owning a home gives a chance of wealth accumulation , it is no sure thing . homes can also drop in value , which makes their support such a big concern for home ##own ##ers . and the method for maintaining property values today is essentially the same as it was in 1936 , though people are less likely to say it out loud : " " if a neighborhood is to retain stability , it


  0%|          | 2/3525 [02:21<66:33:18, 68.01s/it]

...

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] corporate tax in alaska is one of the harsh ##est in the u . s . i think its 31 . 5 % ? and corporate income tax is the 2nd highest ( pa is 9 . 99 % ) too at 9 . 4 % ( matching d . c and exceeding ny ) [SEP]
INFO:tensorflow:input_ids: 101 5971 4171 1999 7397 2003 2028 1997 1996 8401 4355 1999 1996 1057 1012 1055 1012 1045 2228 2049 2861 1012 1019 1003 1029 1998 5971 3318 4171 2003 1996 3416 3284 10


  2%|▏         | 61/3525 [1:13:33<68:11:36, 70.87s/it]

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] " good ! now stay inside , you can ' t handle it out in the cruel cruel world . gee , where do these people come from ? " [SEP]
INFO:tensorflow:input_ids: 101 1000 2204 999 2085 2994 2503 1010 2017 2064 1005 1056 5047 2009 2041 1999 1996 10311 10311 2088 1012 20277 1010 2073 2079 2122 2111 2272 2013 1029 1000 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 


  2%|▏         | 62/3525 [1:14:51<70:13:48, 73.01s/it]

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] well i suppose if he ' s charging with teeth bared and snarl ##ing [SEP]
INFO:tensorflow:input_ids: 101 2092 1045 6814 2065 2002 1005 1055 13003 2007 4091 23485 1998 24845 2075 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [0]:
# from bert_embedding import BertEmbedding
# import mxnet as mx
# ctx = mx.gpu(0)
bert = BertEmbedding(batch_size=64)

In [0]:
for batch in tqdm(range(n_batch)):
    batch_start = batch_size * batch
    batch_end = batch_size * (batch+1)
    batch_file = train[batch_start:batch_end].tolist()
    tmp_matrix = np.zeros((batch_size, 768), dtype='float16')
    for i, matrix in enumerate(bert(batch_file)):
      tmp_matrix[i] = np.mean(np.asarray(matrix[1]), 0)
    if batch > 0:
      tmp_matrix = np.vstack((tmp_matrix, np.load('drive/My Drive/Data/bert_matrix.npy')))
    np.save('drive/My Drive/Data/bert_matrix', tmp_matrix)
    del tmp_matrix, matrix, batch_file
    gc.collect()